### Standard Sample Rate

* What is standard sample rate?

In audio production, a sample rate (or "sampling rate") defines how many times per second a sound is sampled. Technically speaking, it is the frequency of samples used in a digital recording. The standard sample rate used for audio CDs is 44.1 kilohertz (44,100 hertz).

* Why should we resample?

Some of the sound files are sampled at a sample rate of 48000Hz, while most are sampled at a rate of 44100Hz. This means that 1 second of audio will have an array size of 48000 for some sound files, while it will have a smaller array size of 44100 for the others. Once again, we must standardize and convert all audio to the same sampling rate so that all arrays have the same dimensions.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("drive/My Drive/AMHARIC")
os.listdir()

['README.md', 'data', 'kaldi-script', 'lang', 'lm', 'meta_data.csv']

In [3]:
import librosa   #for audio processing
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile #for audio processing
import warnings
warnings.filterwarnings("ignore")

* Create meta data to pass the path of the audios to resample it to 44.1 KHZ

In [8]:
import pandas as pd
meta_data=pd.read_csv("data/train/trsTrain.txt",sep="\t",header=None)
meta_data.rename(columns = {0:'Transcript'}, inplace = True)
meta_data['audio'] = meta_data['Transcript'].apply(lambda x: x.split("</s>")[1].replace("(", "").replace(")", "").strip())
meta_data['Transcript'] = meta_data['Transcript'].apply(lambda x: x.split("</s>")[0])
meta_data['Transcript'] = meta_data['Transcript'].apply(lambda x: x.split("<s>")[1].strip())
meta_data['audio'] = meta_data['audio'].apply(lambda x: "data/train/wav/"+x+".wav")
meta_data

,Transcript,audio
0,ያንደኛ ደረጃ ትምህርታቸው ን ጐንደር ተ ም ረዋል,data/train/wav/tr_1_tr01001.wav
1,የተ ለቀቁት ምርኮኞች በ አካባቢያቸው ሰላማዊ ኑሮ እንዲ ኖሩ የ ትራንስፖ...,data/train/wav/tr_2_tr01002.wav
2,በ አዲስ አበባው ስታዲየም በ ተካሄዱ ት ሁለት ግጥሚያ ዎች በ መጀመሪያ ...,data/train/wav/tr_3_tr01003.wav
3,ወሬው ን ወሬ ያደረጉ ምስጢረ ኞች ናቸው,data/train/wav/tr_4_tr01004.wav
4,ኢትዮጵያዊ ቷ በ ብሄራዊ ባህላዊ አለባበስ ከ አለም አንደኝነት ን ተቀዳጀ ች,data/train/wav/tr_5_tr01005.wav
...,...,...
10870,እንስራ ው ተ ሸነቆረ,data/train/wav/tr_10871_tr09145.wav
10871,ቤዛ ጐረ መሰ መሰለኝ ትእዛዝ አል ቀበል ም አለ,data/train/wav/tr_10872_tr09146.wav
10872,በለጠ ች የ በየነ የ በኩር ልጅ ነች,data/train/wav/tr_10873_tr09147.wav
10873,እንዲያ መሬት አይ ን ካ ኝ ይል የነበረ ሰው በ ድንገት ቆረቆዘ አይደል,data/train/wav/tr_10874_tr09148.wav


# Resampling

In [ ]:
# resampling any loaded audio files to 44.1KHZ 
def resample(df, column):
    sampled_audio = []
    rates = []
    for i in df[column]:
        audio, rate=librosa.load(i, sr=44100)
        sampled_audio.append(audio)
        rates.append(rate)
    
    return sampled_audio, rates
resample(meta_data, 'audio')


# TEST

In [ ]:
import unittest

class TestResampling(unittest.TestCase):
    def test_check_resampling(self):
        sampled_audio, rates = resample(data, 'key')
        expected = [44100, 44100, 44100, 44100, 44100]
        self.assertEqual(rates[8:13], expected)